# Mini-projet Groupe 1 : Charge d'une flotte de véhicules électriques

# Corentin Loirs et Juliette Gerbaux

## 1. Modélisation

### Question 1 :

On suppose que les batteries des voitures sont identiques. Notons le taux de charge relative $\tau\,=\,\frac{q}{q_{max}}$ où $q$ est la charge absolue de la batterie, $q_{max}$ la charge absolue maximale et $P$ la puissance fournie à la batterie. 

On suppose qu'il y a toujours assez de bornes de recharge pour les voitures qui souhaitent être rechargées. On note $n$ le nombre de voitures et pour chaque voiture $j$ le temps $t^0_j$ d'arrivée et $t^f_j$ le temps de départ ainsi que $\tau^0_j$ et $\tau^f_j$ les taux de charge relatifs à l'arrivée et au départ.

Notons $f$ la fonction objectif du problème qui est le coût de recharge lié à la consommation électrique qui dépend de l'intensité $I_j$ qui parcourt chaque borne de recharge $j$ à tout instant.
    
$f(I)\,=\,\int^{t_f}_{t_0}\sum_{j=1}^nI_j(t)\,Up(t)dt$ où $t_0 = \displaystyle \max_{j}(t^0_j)$, $t_f = \displaystyle \max_{j}(t^f_j)$, $U$ la tension du réseau (que l'on suppose fixe égale à $230 V$) et $p(t)$ est le prix de l'électricité en euros/$kWh$.

### Question 2 :

On a : $\frac{dq}{dt}(t) = \alpha\,I(t) = \alpha\,\frac{P(t)}{U}$ d'où : $\frac{d\tau}{dt}(t) =\alpha\, \frac{P(t)}{U*q_{max}} = \alpha\, \frac{I(t)}{q_{max}}$ où $\alpha$ est un facteur lié aux pertes diverses de charge (effet Joule, cablage...) à déterminer expérimentalement. 

### Question 3 :

On va ici traiter de l'évaluation expérimentale de notre modèle, i.e. le calcul par la régression (linéaire dans notre cas, puisque la relation $\frac{dq}{dt}(I)$ est linéaire). On trace d'après les données fournies $\frac{dq}{dt}(t)$ en fonction de $I(t)$.

Etant donné que du bruit a été ajouté aux données et qu'on a accès à $q(t)$ et non $\frac{dq}{dt}(t)$, on va tenter deux méthodes de régression : 
* La première consiste à extrapoler $\frac{dq}{dt}(t)$ d'après les données et tracer cette approximation en fonction de $I(t)$ donné
* La seconde consiste à intégrer $I(t)$ par méthode numérique et tracer $q$ en fonction de  $\int I$.

Dans les deux cas, on aura un coefficient de régression qui vaudra (si tant est que le modèle linéaire est juste) $\frac{\alpha}{q_{max}}$.

In [ ]:
#traitement des données, mise en forme sous un panda
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

table = pd.read_table('donnees-projet-gr1.txt', names=['temps', 'intensité', 'charge_relative'])
table.head()

##### Par le calcul de la dérivée

In [ ]:
### On calcule la dérivée de la charge
table_der = table.copy(deep = True)
table_der['derivee_charge'] = 0
for i in table_der.index :
    if i == 0:
        table_der.iloc[i, 3] = 0
    else :
        table_der.iloc[i, 3] = (table_der.iloc[i-1, 2] - table_der.iloc[i, 2])/(table_der.iloc[i-1, 0] - table_der.iloc[i, 0])
table_der.drop([0], inplace = True) ### On supprime la première ligne car on a pas la dérivée de la charge pour cette ligne.
table_der.head()

In [ ]:
### On crée notre fonction objectif
def f(a):
    s = 0
    for i in table_der.index :
        s += (table_der.loc[i, 'derivee_charge'] - a*table_der.loc[i, 'intensité'] )**2
    return s

In [ ]:
from scipy.optimize import minimize_scalar
res = minimize_scalar(f)
res

On trouve donc que $\frac{d\tau}{dt} = 0.00194 \times I$ avec une erreur de 0.00203.

In [ ]:
plt.scatter(table_der['intensité'], table_der['derivee_charge'])
x = np.linspace(-4, 1)
plt.plot(x, 0.0019426071168044741*x, 'g')

Remarquons d'abord que cette régression ne semble pas très précise: la méthode des moindres carrés fait passer la droite par la moyenne quadratique des bruits. Etant donné que le bruit a été rajouté artificiellement, on peut se douter que la moyenne quadratique donne une bonne indication de la valeur réelle. Cependant, si l'on avait un bruit pondéré (par exemple avec beaucoup de marge vers le bas) ou borné, la valeur du coefficient de régression inspirerait peut de confiance.

Pour valider ce résultat (ou l'infirmer), on va passer par une méthode d'intrégration.

##### Par intégration

In [ ]:
### On intègre I par la méthode des trapèzes
table_int = table.copy(deep = True)
table_int['int_I'] = 0
for i in table_int.index :
    if i == 0 : ### Calcul de l'intégrale entre t0 et t1
        table_int.iloc[i, 3] = (table_int.iloc[i+1, 0] - table_int.iloc[i, 0])*(table_int.iloc[i, 1] + table_int.iloc[i, 1])/2
    elif i!=298 : ### On calcule l'intégrale de I entre ti et t(i+1) et on ajoute la valeur de l'intégrale entre t0 et ti
        table_int.iloc[i, 3] = table_int.iloc[i-1,3]+(table_int.iloc[i+1, 0] - table_int.iloc[i, 0])*(table_int.iloc[i, 1] + table_int.iloc[i, 1])/2
    else : ### On ne peut pas caluler la dernière intégrale
        table_int.iloc[i, 3] = 0
table_int.drop([298], inplace = True) ###La dernière valeur n'a pas de sens.
table_int.head()

In [ ]:
plt.scatter(table_int['int_I'], table_int['charge_relative'])

In [ ]:
### On crée notre nouvelle fonction objectif
def g(a):
    s = 0
    for i in  table_int.index :
        s += (table_int.loc[i, 'charge_relative'] - a*table_int.loc[i, 'int_I'] - table_int.loc[0, 'charge_relative'])**2
    return s

In [ ]:
from scipy.optimize import minimize_scalar
res = minimize_scalar(g)
res

In [ ]:
plt.scatter(table_int['int_I'], table_int['charge_relative'])
x = np.linspace(-190, 0)
plt.plot(x, 0.0019426071168044741*x + table_int.loc[0, 'charge_relative'], 'g')

Ce graphe est nettement plus satisfaisant: on retrouve bien une relation affine de la forme $ q = a \int I+q_0$.
On retrouve le même coefficient directeur, qui vaut environ 0.0020. 

On valide donc le modèle proposé, soit $\frac{d\tau}{dt}\,=\,\frac{\alpha}{q_{max}}\times I$ avec $\frac{\alpha}{q_{max}}=0,002$

### Question 4 :

#### Contraintes du problème

Nous avons donc la fonction objectif du problème $f$ qu'il faut minimiser en fonction du vecteur $I$ des instensités. Ecrivons à présent les contraintes du porblème. 

- $I_j(t) = 0$ pour $t \notin [t_j^0, t_j^f]$ pour tout $j$ : la voiture ne peut pas charger si elle n'est pas là.
- $\tau_j(t) \in [0, 1]$ pour tout $t$ et tout $j$ : le taux de charge relatif est compris entre 0 et 1. (deux conditions en une)
- $\tau_j(t_j^0) = \tau^0_j$ pour tout $j$ : le taux de charge de la voiture $j$ lorsque la voiture arrive est égal à la charge initial.
- $\tau_j(t_j^f) \geq \tau^f_j$ pour tout $j$ : le taux de charge de la voiture $j$ lorsque la voiture part est supérieur ou égal au taux de charge final voulu.
- $\sum_{j=1}^n I_j(t)U \leq P_{max}$ pout tout $t$ : la puissance fournie à tout instant $t$ aux voitures ne dépasse pas la puissance maximale $P_{max}$ du réseau. 
- $\frac{d\tau_j(t)}{dt}\,=\,\frac{\alpha}{q_{max}}\times I_j(t)$ avec $\frac{\alpha}{q_{max}}=0,002$ pour tout $t$ et tout $j$.

#### Version discrète du problème

On peut ensuite passer à une version discrétisée du problème pour avoir un problème d'optimisation sous contraintes en dimension finie : 

On commence par discrétiser le temps en $N$ pas. On pose $T_0 = t_0$ et $T_k = T_0 + k\times \Delta T$ où $\Delta T = \frac{t_f - t_0}{N}$ de sorte que $T_N = t_f$. On choisit de réécrire les temps d'arrivée et de départ d'une voiture $j$, $t_j^0 = T_{k_j^0}$ et $t_j^f = T_{k_j^f}$.

Ensuite, on reécrit notre fonction objectif : $f(I) = \sum_{k=0}^{N-1} \sum_{j=1}^n I_j(T_k)\times U \times p(T_k)\Delta T$ (on a intégré selon la méthodes des rectangles à gauche).

Et enfin, nos contraintes :
- $I_j(T_k) = 0$ pour $T_k \notin [T_{k_j^0},T_{k_j^f}]$ ou encore pour tout $k \notin [\![k_j^0, k_j^f ]\!]$ et pour tout $j$: la voiture ne peut pas charger si elle n'est pas là.
- $\tau_j(T_k) \in [0, 1]$ pour tout $k$ et tout $j$ : le taux de charge relatif est compris entre 0 et 1. (deux conditions en une)
- $\tau_j(T_{k_j^0}) = \tau^0_j$ pour tout $j$ : le taux de charge de la voiture $j$ lorsque la voiture arrive est égal à la charge initial.
- $\tau_j(T_{k_j^f}) \geq \tau^f_j$ pour tout $j$: le taux de charge de la voiture $j$ lorsque la voiture part est supérieur ou égal au taux de charge final voulu.
- $\sum_{j=1}^n I_j(T_k)U \leq P_{max}$ pout tout $k$ : la puissance fournie à tout instant aux voitures ne dépasse pas la puissance maximale $P_{max}$ du réseau. 
- $\tau_j(T_{k+1})\,=\,\tau_j(T_{k}) + \frac{\alpha}{q_{max}}\times I_j(T_k)$ avec $\frac{\alpha}{q_{max}}=0,002$ pour tout $k$ et tout $j$.

##### Version avec des matrices

On peut aussi réécrire ce problème en utilisant trois matrices et la norme 1 sur $R^n$. 

On utilise la matrice I des intensités telle que $I_{j,k} = I_j(T_k)$, la matrice colonne P des prix telle $P_k = p(T_k)$ et la matrice $\tau$ des charges relatives telle que $\tau_{j,k} = \tau_j(T_k)$.

La fonction objectif est alors : $f(I) = U\times \Delta T \times||I\times P||_1$ (cette relation n'est valable que lorsque tous les coefficients de $I$ sont positifs).

Et les contraintes s'écrivent :
- $I_{j,k} = 0$ pour tout $j$ et pour tout $k \notin [\![k_j^0, k_j^f ]\!]$ 
- $\tau_{j,k} \in [0, 1]$ pour tout $k$ et tout $j$ 
- $\tau_{j,k_j^0} = \tau^0_j$ pour tout $j$ 
- $\tau_{j,k_j^f} \geq \tau^f_j$ pout tout $j$
- $||U\times I_k||_1\leq P_{max}$ pout tout $k$ où $I_k$ est la $k^{ème}$ colonne de $I$.
- $\tau_{j,k+1}\,=\,\tau_{j,k} + \frac{\alpha}{q_{max}}\times I_{j,k}$ avec $\frac{\alpha}{q_{max}}=0,002$ pour tout $k$ et tout $j$.

## 2. Etude et résolution numérique 

A partir de maintenant, on choisit de garder les notations et la modélisation de la correction. 

On discrètise le temps en N pas. Pour une seule voiture, la fonction à minimiser est : $f(w) = p^T\times w$ où $w$ est un vecteur de $\mathbb{R}^N$ qui représente la puissance fournie à la voiture au cours du temps et $p$ est la matrice des prix. Les contraintes sont $\Delta q = b_0 1^T_{[n_0, n_f]} w$ et $0 \leq w \leq w_{max}$.

### Question 1 :

La fonction f est bien convexe car linéaire. Les contraintes sont aussi linéaires donc les contraintes sont linéaires. La condition $0 \leq w \leq w_{max} $ implique que l'on travaille sur un fermé donc que la fonction $f$ (qui est continue) admet bien un minumum et que celui-ci est unique car la fonction $f$ est convexe. 

On peut réécrire le problème sous forme d'un problème quadratique avec contraintes affines : 
- $f(w) = w^T\times p$
- $ c(w) = Aw - b \leq 0 $est une matrice à $2N+1$ lignes et $N$ colonnes $A =$ \begin{bmatrix} -I_N\\ I_N \\ 1^T_{[n_0, n_f]} \end{bmatrix} et $b$ est une matrice colonne à $2N+1$ linges : $b=$ \begin{bmatrix} 0\\ ... \\0 \\ w_{max} \\ ... \\ w_{max} \\ \frac{\Delta q}{b_0} \end{bmatrix}

En effet, on peut tranformer la contrainte $\Delta q = b_0 1^T_{[n_0, n_f]} w$ en une contrainte inégalité $ b_0 1^T_{[n_0, n_f]} w -\Delta q \leq 0 $ qui sera nécessairement active pour minimiser $f$ car $p$ et $w$ sont positifs donc on n'a pas intéret à charger plus que ce qui est demandé.

On peut donc réssoudre ce problème avec l'algorithme des contraintes actives.

### Question 2 :

On choisit de discrétiser le temps N = 1440 pas de temps de 1 minute. Le prix horraire du kWh est en première approximation égal à 0.15 euros. On considère que la voiture arrive à $n_0 = 540$ minutes et repart à $n_f = 720$ minutes. On prend $w_{max} = 20 kW$. On choisit aussi $\frac{\Delta q}{b_0} = \Delta \tau \frac{q_{max}}{b_0} = \frac{0.7}{0.002}$.

In [ ]:
import numpy as np 
N = 1440 
p = np.array([0.15]*N)
def f(w):
    return w.T & p

n0 = 540
nf = 720
I_N = np.identity(N)
matrice_1 = np.zeros((1,N))
for i in range(N):
    if n0 <= i <= nf :
        matrice_1[0, i] = 1
A = np.concatenate((-I_N, I_N))
A = np.concatenate((A, matrice_1))

b = np.zeros(2*N+1)
for i in range(2*N+1):
    if i >= 1440 :
        b[i] = 20
    elif i == 2*N:
        b[i] = 0.7/0.002
        
def c(w):
    return A&w - b

## 3. Etude avancée

### Question 3 : 

Avec $n$ voitures, on peut utiliser une méthode de décomposition-coordination. En effet, la fonction objectif est $f(w_1, ... , w_n) = \sum_{i=1}^n p^T w_i = \sum_{i=1}^n f_i(w_i)$ où $w_i$ est la matrice de la puissance fournie à la voiture $i$ et $f_i(w_i) = p^T w_i$.

Comme dans la partie précédente, on peut réécrire la contrainte $\Delta q_i = b_0 1^T_{[n_{i,0}, n_{i,f}]} w_i$ en tant que contrainte inégalité en considérant qu'elle sera forcément active puisque $w_i$ et $p$ sont positifs : $b_0 1^T_{[n_{i,0}, n_{i,f}]} w_i - \Delta q_i  \leq 0$. Cette dernière contrainte que l'on va appeler contrainte $\alpha$ s'écrit donc $c^\alpha(w_1, ... w_n) = D w - Q \leq 0$ où $w$ est la matrice colonne des concaténations des $w_i$, $D$ est la matrice te taille $n\times nN$ définie par blocs : 
\begin{equation}D =\begin{bmatrix} D_1 & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & D_n \end{bmatrix} \end{equation}
où $ D_i = 1^T_{[n_{i,0}, n_{i,f}]} $
et $Q$ est la matrice colonne des $\frac{\Delta q_i}{b_0}$. Cette contrainte se réécrit $c^\alpha(w_1, ... w_n) = \sum_{i=1}^n D \phi_i(w_i) - \psi_i(Q) = \sum_{i=1}^n c_i^\alpha(w_i) \leq 0$ où $c_i^\alpha(w_i) = D \phi_i(w_i) - \psi_i(Q)$ et où $\phi_i(w_i)$ est une matrice colonne à $nN$ lignes égale à 
\begin{bmatrix} 0 \\ ... \\ 0 \\ w_i \\ 0 \\ ... \\ 0 \end{bmatrix} et 
\begin{equation} \psi_i(Q) = \begin{bmatrix} 0 \\ ... \\ 0 \\ \frac{\Delta q_i}{b_0} \\ 0 \\ ... \\ 0 \end{bmatrix} \end{equation}

La contrainte $\beta : -w_i \leq 0$ s'écrit $c^\beta (w_1, ..., w_n) = - I_{nN} w \leq 0$  ou encore $c^\beta(w) = \sum_{i=1}^n c_i^\beta(w_i)$ où $c_i^\beta(w_i) = -  I_{nN} \phi_i(w_i)$.

La contrainte $\gamma: \sum_{i=1}^n w_i - w_{max} \leq 0$ s'écrit $c^\gamma (w_1, ..., w_n) = C w - W \leq 0$ où $C$ est la matrice est une matrice à $N$ lignes et $nN$ colonnes : 
\begin{equation} C= \begin{bmatrix} I_N &... &I_N \end{bmatrix} \end{equation}
et $W$ est la matrice colonne à $N$ lignes où chaque coefficient est égal à $w_{max}$. On a donc : $c^\gamma (w_1, ..., w_n) = \sum_{i=1}^n c_i^\gamma (w_i)$ où $c_i^\gamma (w_i) = C\phi_i(w_i) - \frac{1}{n}W$.

La contrainte globale sur $w$ s'écrit donc : $c(w) = Aw-b$ où \begin{equation} A = \begin{bmatrix} D \\ I_{nN} \\ C \end{bmatrix} \end{equation} et \begin{equation} b = \begin{bmatrix} Q \\ 0 \\ ... \\ 0 \\ W  \end{bmatrix} \end{equation}
($A$ est une matrice à $n  + nN + N$ lignes et $nN$ colonnes, $b$ est une matrice colonne à $n  + nN + N$ lignes)

Cette contrainte se réécrit : $c(w) = \sum_{i=1}^n A\phi_i(w_i) -b_i = \sum_{i=1}^n c_i(w_i) $ où $c_i(w_i) =  A\phi_i(w_i) -b_i$ et où  \begin{equation}b_i = \begin{bmatrix} \psi_i(Q) \\ 0 \\ ... \\ 0 \\ \frac{1}{n}W \end{bmatrix} \end{equation}

Le problème s'écrit donc : 
\begin{equation} f(w) = \sum_{i=1}^n f_i(w_i) , f_i(w_i) = p^T w_i \end{equation}
\begin{equation} c(w) = \sum_{i=1}^n c_i(w_i) , c_i(w_i) = A\phi_i(w_i) -b_i \end{equation}
Et on peut utiliser une méthode de décomposition-coordination.

### Question 4 :

In [ ]:
N = 1440 
n = 2
p = np.array([0.15]*N)

def f(i, w):
    return w.T & p

n0 = np.array([ , ])
nf = np.array([ , ])
I_nN = np.identity(n*N)
D = np.zeros((n,N))
for j in range(n):
    for i in range(N):
        if n0[j] <= i <= nf[j] :
            matrice_1[j, i] = 1
C = np.concatenate((np.identity(N), np.identity(N)), axis = 1)
for j in range(n-2):
    C = np.concatenate((C, np.identity(N)), axis = 1)
A = np.concatenate((D, I_nN))
A = np.concatenate((A, C))

b = np.zeros((n + n*N + N, n))
for j in range(n):
    for i in range(2*N+1):
        if i ==j:
            b[i, j] =
        elif i >= n + n*N :
            b[i, j] = 20/n
            
        
def c(i ,w):
    phi_w = np.zeros(n*N)
    for j in range(N):
        phi_w[i + j] = w[i]
    return A&phi_w - (b.T[i]).T